<a href="https://colab.research.google.com/github/josealprestes/ia_tests/blob/main/google_slides_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Inicialize o cliente da API do Google Slides
slides_service = build('slides', 'v1', cache_discovery=False)


In [7]:
titulo = "Letramento Digital"
subtitulos = [
    "Introdução",
    "Componentes do Letramento Digital",
    "Importância do Letramento Digital",
    "Desafios e Soluções",
    "Conclusão",
]

slides = [
    "O letramento digital é a habilidade de buscar, entender, avaliar e criar informações usando tecnologias digitais.",
    "O letramento digital inclui habilidades técnicas, cognitivas e sociais para navegar no mundo digital.",
    "O letramento digital é crucial para o sucesso no mundo atual, tanto no ambiente de trabalho quanto na vida cotidiana.",
    "Os desafios do letramento digital incluem a desigualdade no acesso à tecnologia e a necessidade de melhorar a educação digital.",
    "Promover o letramento digital é essencial para capacitar as pessoas a participar plenamente na sociedade digital.",
]


In [9]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
service = build('slides', 'v1', cache_discovery=False)


In [20]:
# Função para criar a apresentação
def create_presentation(title, subtitle, subtitles, content):
    try:
        # Criar o cliente do Google Slides
        service = build('slides', 'v1', credentials=creds)
        
        # Criar uma nova apresentação
        body = {
            'title': title
        }
        presentation = service.presentations().create(body=body).execute()
        presentation_id = presentation['presentationId']
        print(f"Apresentação criada com o ID: {presentation_id}")

        # Adicionar o tema à apresentação
        requests = [
            {
                'updatePresentationProperties': {
                    'properties': {
                        'themeId': theme_id
                    },
                    'fields': 'themeId'
                }
            }
        ]
        response = service.presentations().batchUpdate(presentationId=presentation_id, body={'requests': requests}).execute()

        # Adicionar o primeiro slide à apresentação
        slide_requests = [
            {
                'createSlide': {
                    'insertionIndex': '0',
                    'slideLayoutReference': {
                        'predefinedLayout': 'TITLE_AND_TWO_COLUMNS'
                    }
                }
            }
        ]
        slide_response = service.presentations().batchUpdate(presentationId=presentation_id, body={'requests': slide_requests}).execute()

        # Adicionar o título à primeira slide
        title_request = {
            'insertText': {
                'objectId': presentation['slides'][0]['pageElements'][0]['objectId'],
                'insertionIndex': 0,
                'text': title
            }
        }
        response = service.presentations().batchUpdate(presentationId=presentation_id, body={'requests': [title_request]}).execute()

        # Adicionar o subtítulo à primeira slide
        subtitle_request = {
            'insertText': {
                'objectId': presentation['slides'][0]['pageElements'][1]['objectId'],
                'insertionIndex': 0,
                'text': subtitle
            }
        }
        response = service.presentations().batchUpdate(presentationId=presentation_id, body={'requests': [subtitle_request]}).execute()

        # Adicionar os demais slides à apresentação
        for i in range(len(subtitles)):
            slide_requests = [
                {
                    'createSlide': {
                        'insertionIndex': str(i + 1),
                        'slideLayoutReference': {
                            'predefinedLayout': 'TITLE_AND_BODY'
                        }
                    }
                }
            ]
            slide_response = service.presentations().batchUpdate(presentationId=presentation_id, body={'requests': slide_requests}).execute()

            # Adicionar o título do slide
            title_request = {
                'insertText': {
                    'objectId': f'slide{i + 1}_title',
                    'insertionIndex': 0,
                    'text': subtitles[i]
                }
            }
            response = service.presentations().batchUpdate(presentationId=presentation_id, body={'requests': [title_request]}).execute()

            # Adicionar o conteúdo do slide
            body_request = {
                'insertText': {
                    'objectId': f'slide{i + 1}_body',
                    'insertionIndex': 0,
                    'text': content[i]
                }
            }
            response = service.presentations().batchUpdate(presentationId=presentation_id, body={'requests': [body_request]}).execute()

        print("Apresentação criada com sucesso.")
    except HttpError as error:
        print(f"Ocorreu um erro: {error}")
       
